In [1]:
#!/usr/bin/env python
# coding: utf-8

# the original is from https://datawhalechina.github.io/hello-agents/#/./README
# if you can, please give a star to them

import os
import json
import re
from openai import OpenAI
from dotenv import load_dotenv
from typing import List, Dict, Any, Callable
from serpapi import SerpApiClient

# Load environment variables
load_dotenv()

# --- 1. LLM Client Definition ---

class HelloAgentsLLM:
    """
    Custom LLM Client for 'Hello Agents'.
    Compatible with OpenAI-style APIs with streaming support.
    """
    def __init__(self, model: str = None, apiKey: str = None, baseUrl: str = None, timeout: int = None):
        self.model = model or os.getenv("LLM_MODEL_ID")
        apiKey = apiKey or os.getenv("LLM_API_KEY")
        baseUrl = baseUrl or os.getenv("LLM_BASE_URL")
        timeout = timeout or int(os.getenv("LLM_TIMEOUT", 60))
        
        if not all([self.model, apiKey, baseUrl]):
            raise ValueError("Model ID, API Key, and Base URL must be provided or defined in .env.")

        self.client = OpenAI(api_key=apiKey, base_url=baseUrl, timeout=timeout)

    def think(self, messages: List[Dict[str, str]], temperature: float = 0) -> str:
        """Call the LLM and return the response text."""
        print(f"🧠 Calling {self.model}...")
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                temperature=temperature,
                stream=True,
            )
            
            collected_content = []
            for chunk in response:
                content = chunk.choices[0].delta.content or ""
                print(content, end="", flush=True)
                collected_content.append(content)
            print() 
            return "".join(collected_content)
        except Exception as e:
            print(f"❌ Error calling LLM API: {e}")
            return None

# --- 2. Tool Definitions ---

def google_search(query: str) -> str:
    """A search tool based on SerpApi."""
    print(f"🔍 Executing [SerpApi] Search: {query}")
    try:
        api_key = os.getenv("SERPAPI_API_KEY")
        if not api_key:
            return "Error: SERPAPI_API_KEY not found in environment."

        params = {
            "engine": "google",
            "q": query,
            "api_key": api_key,
            "gl": "us",
            "hl": "en",
        }
        
        client = SerpApiClient(params)
        results = client.get_dict()
        
        if "answer_box" in results and "answer" in results["answer_box"]:
            return results["answer_box"]["answer"]
        if "knowledge_graph" in results and "description" in results["knowledge_graph"]:
            return results["knowledge_graph"]["description"]
        if "organic_results" in results:
            snippets = [
                f"[{i+1}] {res.get('title')}: {res.get('snippet')}"
                for i, res in enumerate(results["organic_results"][:3])
            ]
            return "\n\n".join(snippets)
        
        return f"No results found for '{query}'."
    except Exception as e:
        return f"Search error: {e}"

class ToolExecutor:
    """Manages and executes tools."""
    def __init__(self):
        self.tools: Dict[str, Dict[str, Any]] = {}

    def register_tool(self, name: str, description: str, func: Callable):
        self.tools[name] = {"description": description, "func": func}
        print(f"✅ Tool '{name}' registered.")

    def execute(self, name: str, tool_input: str) -> str:
        """Central execution method used by the Agent."""
        if name not in self.tools:
            return f"Error: Tool '{name}' not found."
        return self.tools[name]["func"](tool_input)

    def get_available_tools_description(self) -> str:
        return "\n".join([f"- {name}: {info['description']}" for name, info in self.tools.items()])

# --- 3. ReAct Agent Core ---

REACT_PROMPT_TEMPLATE_ENGLISH = """
# System Role
You are an advanced AI Agent capable of iterative reasoning and tool usage. 
Your goal is to solve the user's request using a Thought -> Action -> Observation loop.

# Available Tools
{tools}

# Response Format (STRICT JSON)
You must respond ONLY with a valid JSON object. Do not include any text outside the JSON block. 
The JSON must follow this exact structure:
{{
  "thought": "Briefly explain your reasoning and next step.",
  "action": {{
    "name": "ToolName" or "Finish",
    "input": "Specific tool input OR the final detailed answer."
  }}
}}

# Rules & Constraints
1. **Single Action**: Only one action per turn.
2. **Finish Condition**: Use "Finish" when you have enough info.
3. **No Preamble**: Start directly with {{.

# Execution Task
Question: {question}
History: 
{history}
"""

class ReActAgentEnglish:
    def __init__(self, llm_client: HelloAgentsLLM, tool_executor: ToolExecutor, max_steps: int = 5):
        self.llm_client = llm_client
        self.tool_executor = tool_executor
        self.max_steps = max_steps
        self.history = []

    def run(self, question: str):
        self.history = [] 
        current_step = 0

        while current_step < self.max_steps:
            current_step += 1
            print(f"\n--- Step {current_step} ---")

            tools_desc = self.tool_executor.get_available_tools_description()
            history_str = "\n".join(self.history) if self.history else "No history yet."
            
            prompt = REACT_PROMPT_TEMPLATE_ENGLISH.format(
                tools=tools_desc,
                question=question,
                history=history_str
            )

            response_text = self.llm_client.think([{"role": "user", "content": prompt}])
            data = self._parse_json_safely(response_text)
            
            if not data:
                print("❌ Error: Failed to parse JSON.")
                break

            thought = data.get("thought")
            action = data.get("action", {})
            a_name = action.get("name")
            a_input = action.get("input")

            print(f"Thought: {thought}")

            if a_name == "Finish":
                print(f"\n🎉 Final Answer: {a_input}")
                return a_input

            # Execute tool
            observation = self.tool_executor.execute(a_name, a_input)
            print(f"👀 Observation: {observation}")
            
            # Update History
            self.history.append(f"Step {current_step} - Thought: {thought}")
            self.history.append(f"Step {current_step} - Action: {a_name}[{a_input}]")
            self.history.append(f"Step {current_step} - Observation: {observation}")

        print("Reached max steps.")
        return None

    def _parse_json_safely(self, text: str):
        try:
            return json.loads(text)
        except:
            match = re.search(r"(\{.*\})", text, re.DOTALL)
            if match:
                try: return json.loads(match.group(1))
                except: return None
        return None

# --- 4. Execution ---

# if __name__ == "__main__":
#     # Ensure your .env has: LLM_MODEL_ID, LLM_API_KEY, LLM_BASE_URL, SERPAPI_API_KEY
#     llm = HelloAgentsLLM()
    
#     executor = ToolExecutor()
#     executor.register_tool(
#         "Search", 
#         "A search engine for real-time facts and current events.", 
#         google_search
#     )
    
#     agent = ReActAgentEnglish(llm_client=llm, tool_executor=executor)
#     user_query = "Which is the newest version of huawei smartphone? what are its features?"
#     agent.run(user_query)
# # modelscope deepseek v3.2, 4 steps

✅ Tool 'Search' registered.

--- Step 1 ---
🧠 Calling deepseek-ai/DeepSeek-V3.2...
{
  "thought": "The user is asking for the newest Huawei smartphone and its features. I need to get current, up-to-date information, as smartphone models are frequently updated. I'll use the search tool to find the latest Huawei release.",
  "action": {
    "name": "Search",
    "input": "newest Huawei smartphone model latest version 2024 features"
  }
}
Thought: The user is asking for the newest Huawei smartphone and its features. I need to get current, up-to-date information, as smartphone models are frequently updated. I'll use the search tool to find the latest Huawei release.
🔍 Executing [SerpApi] Search: newest Huawei smartphone model latest version 2024 features
👀 Observation: [1] HUAWEI Phones: Explore the latest technologies in smartphone with HUAWEI, and check out the HUAWEI Mate series, HUAWEI Pura series and HUAWEI nova series.

[2] The best Huawei phones in 2025: Huawei Mate X6 - The latest 

In [2]:
# if __name__ == "__main__":
#     # Ensure your .env has: LLM_MODEL_ID, LLM_API_KEY, LLM_BASE_URL, SERPAPI_API_KEY
#     llm = HelloAgentsLLM(
#         model=os.getenv("MS_DS_LLM_MODEL_ID"),
#         baseUrl=os.getenv("MS_DS_LLM_BASE_URL"),
#         apiKey=os.getenv("MS_DS_LLM_API_KEY"),
#     )
    
#     executor = ToolExecutor()
#     executor.register_tool(
#         "Search", 
#         "A search engine for real-time facts and current events.", 
#         google_search
#     )
    
#     agent = ReActAgentEnglish(llm_client=llm, tool_executor=executor)
#     user_query = "Which is the title of the newest blog of athropic?"
#     agent.run(user_query)

✅ Tool 'Search' registered.

--- Step 1 ---
🧠 Calling deepseek-ai/DeepSeek-V3.2...
{
  "thought": "The user is asking for the title of the newest blog post from Anthropic. To answer this, I need to find the most recent blog post on Anthropic's official blog. This requires a web search to get real-time, up-to-date information.",
  "action": {
    "name": "Search",
    "input": "Anthropic newest blog post"
  }
}
Thought: The user is asking for the title of the newest blog post from Anthropic. To answer this, I need to find the most recent blog post on Anthropic's official blog. This requires a web search to get real-time, up-to-date information.
🔍 Executing [SerpApi] Search: Anthropic newest blog post
👀 Observation: [1] Newsroom: Introducing Claude Sonnet 4.5 · Introducing Claude Haiku 4.5 · Anthropic raises $13B Series F at $183B post-money valuation · Thoughts on America's AI Action Plan.

[2] Engineering \ Anthropic: Anthropic is an AI safety and research company that's working to bui